In [1]:
import numpy as np
import cv2
from numpy import expand_dims
from skimage.transform import resize
from keras.models import load_model, Model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

In [2]:
model = load_model('face_recognition_model.h5')

In [3]:
class BoundBox:
    def __init__(self, xmin, ymin, xmax, ymax, objness = None, classes = None):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax
        self.objness = objness
        self.classes = classes
        self.label = -1
        self.score = -1

    def get_label(self):
        if self.label == -1:
            self.label = np.argmax(self.classes)

        return self.label

    def get_score(self):
        if self.score == -1:
            self.score = self.classes[self.get_label()]
            
        return self.get_score

In [4]:
def _sigmoid(x):
    return 1. /(1. + np.exp(-x))

In [5]:
def decode_netout(netout, anchors, obj_thresh, net_h, net_w):
    grid_h, grid_w = netout.shape[:2]
    nb_box = 3
    netout = netout.reshape((grid_h, grid_w, nb_box, -1))
    nb_class = netout.shape[-1] - 5
    boxes = []
    netout[..., :2]  = _sigmoid(netout[..., :2])
    netout[..., 4:]  = _sigmoid(netout[..., 4:])
    netout[..., 5:]  = netout[..., 4][..., np.newaxis] * netout[..., 5:]
    netout[..., 5:] *= netout[..., 5:] > obj_thresh
 
    for i in range(grid_h*grid_w):
        row = i / grid_w
        col = i % grid_w
        for b in range(nb_box):
            # 4th element is objectness score
            objectness = netout[int(row)][int(col)][b][4]
            if(objectness.all() <= obj_thresh): continue
            # first 4 elements are x, y, w, and h
            x, y, w, h = netout[int(row)][int(col)][b][:4]
            x = (col + x) / grid_w # center position, unit: image width
            y = (row + y) / grid_h # center position, unit: image height
            w = anchors[2 * b + 0] * np.exp(w) / net_w # unit: image width
            h = anchors[2 * b + 1] * np.exp(h) / net_h # unit: image height
            # last elements are class probabilities
            classes = netout[int(row)][col][b][5:]
            box = BoundBox(x-w/2, y-h/2, x+w/2, y+h/2, objectness, classes)
            boxes.append(box)
    return boxes

In [6]:
def correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w):
    new_w, new_h = net_w, net_h
    for i in range(len(boxes)):
        x_offset, x_scale = (net_w - new_w)/2./net_w, float(new_w)/net_w
        y_offset, y_scale = (net_h - new_h)/2./net_h, float(new_h)/net_h
        boxes[i].xmin = int((boxes[i].xmin - x_offset) / x_scale * image_w)
        boxes[i].xmax = int((boxes[i].xmax - x_offset) / x_scale * image_w)
        boxes[i].ymin = int((boxes[i].ymin - y_offset) / y_scale * image_h)
        boxes[i].ymax = int((boxes[i].ymax - y_offset) / y_scale * image_h)

In [7]:
def _interval_overlap(interval_a, interval_b):
    x1, x2 = interval_a
    x3, x4 = interval_b
    if x3 < x1:
        if x4 < x1:
            return 0
        else:
            return min(x2,x4) - x1
    else:
        if x2 < x3:
             return 0
        else:
            return min(x2,x4) - x3

In [8]:
def bbox_iou(box1, box2):
    intersect_w = _interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
    intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])
    intersect = intersect_w * intersect_h
    w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
    w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin
    union = w1*h1 + w2*h2 - intersect
    return float(intersect) / union

In [9]:
def do_nms(boxes, nms_thresh):
    if len(boxes) > 0:
        nb_class = len(boxes[0].classes)
    else:
        return
    for c in range(nb_class):
        sorted_indices = np.argsort([-box.classes[c] for box in boxes])
        for i in range(len(sorted_indices)):
            index_i = sorted_indices[i]
            if boxes[index_i].classes[c] == 0:
                continue
            for j in range(i+1, len(sorted_indices)):
                index_j = sorted_indices[j]
                if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_thresh:
                    boxes[index_j].classes[c] = 0

In [10]:
# load and prepare an image
def load_image_pixels(image, shape):
    width, height, _ = image.shape
    # convert to numpy array
    image = img_to_array(image)
    # scale pixel values to [0, 1]
    image = image.astype('float32')
    image /= 255.0
    # add a dimension so that we have one sample
    image = expand_dims(image, 0)
    return image, width, height

In [11]:
# get all of the results above a threshold
def get_boxes(boxes, labels, thresh):
    v_boxes, v_labels, v_scores = list(), list(), list()
    # enumerate all boxes
    for box in boxes:
        # enumerate all possible labels
        for i in range(len(labels)):
            # check if the threshold for this label is high enough
            if box.classes[i] > thresh:
                v_boxes.append(box)
                v_labels.append(labels[i])
                v_scores.append(box.classes[i]*100)
                # don't break, many labels may trigger for one box
    return v_boxes, v_labels, v_scores

In [12]:
# draw all results
def draw_boxes(img, v_boxes, v_labels, v_scores):
    for i in range(len(v_boxes)):
        # retrieving the coordinates from each bounding box
        box = v_boxes[i]
        # get coordinates
        y1, x1, y2, x2 = box.ymin, box.xmin, box.ymax, box.xmax
        # Ending coordinate: top left corner of rectangle 
        start_point = (x1+75, y1-105) 
        # Ending coordinate: bottom right corner of rectangle 
        end_point = (x2+75, y2-145) 
        # Red color in BGR 
        color = (0, 0, 255) 
        # Line thickness
        thickness = 2
        # font 
        font = cv2.FONT_HERSHEY_PLAIN 
        # fontScale 
        fontScale = 1.5
        #create the shape
        img = cv2.rectangle(img, start_point, end_point, color, thickness) 
        # draw text and score in top left corner
        label = "%s (%.3f)" % (v_labels[i], v_scores[i])
        img = cv2.putText(img, label, start_point, font, fontScale, color, thickness, cv2.LINE_AA)
    return img

In [13]:
# define the anchors
anchors = [[116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]]
    
# define the probability threshold for detected objects
class_threshold = 0.6

# define the labels
labels = ["person face"]

In [14]:
cam=cv2.VideoCapture(0)

while True:
#read the cam video and saving it in image    
    _,photo=cam.read()
    
    input_w, input_h = 416, 416
    image, image_w, image_h = load_image_pixels(photo,(input_w, input_h))

    yhat = model.predict(image)
    
    # summarize the shape of the list of arrays
    print([a.shape for a in yhat])
    
    boxes = list()
    for i in range(len(yhat)):
    # decode the output of the network
        boxes += decode_netout(yhat[i][0], anchors[i], class_threshold, input_h, input_w)

    correct_yolo_boxes(boxes, image_h, image_w, input_h, input_w)
    
    # suppress non-maximal boxes
    do_nms(boxes, 0.5)

    # get the details of the detected objects
    v_boxes, v_labels, v_scores = get_boxes(boxes, labels, class_threshold)
    
    # summarize what we found
    for i in range(len(v_boxes)):
        print(v_labels[i], v_scores[i])
        
    img=draw_boxes(photo, v_boxes, v_labels, v_scores)
    
    # show the plot
    cv2.imshow("image",img)
    print('press q to exit')
    print('.......') 
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
        
cam.release()
cv2.destroyAllWindows()

[(1, 15, 20, 18), (1, 30, 40, 18), (1, 60, 80, 18)]
person face 99.0736722946167
press q to exit
.......
[(1, 15, 20, 18), (1, 30, 40, 18), (1, 60, 80, 18)]
person face 99.65399503707886
press q to exit
.......
[(1, 15, 20, 18), (1, 30, 40, 18), (1, 60, 80, 18)]
person face 98.3755111694336
press q to exit
.......
[(1, 15, 20, 18), (1, 30, 40, 18), (1, 60, 80, 18)]
person face 96.38136625289917
press q to exit
.......
[(1, 15, 20, 18), (1, 30, 40, 18), (1, 60, 80, 18)]
person face 97.63645529747009
press q to exit
.......
[(1, 15, 20, 18), (1, 30, 40, 18), (1, 60, 80, 18)]
person face 98.23812246322632
press q to exit
.......
[(1, 15, 20, 18), (1, 30, 40, 18), (1, 60, 80, 18)]
person face 99.83389377593994
press q to exit
.......
[(1, 15, 20, 18), (1, 30, 40, 18), (1, 60, 80, 18)]
person face 93.33714842796326
press q to exit
.......
[(1, 15, 20, 18), (1, 30, 40, 18), (1, 60, 80, 18)]
person face 98.89600276947021
press q to exit
.......
